<a href="https://colab.research.google.com/github/SoumapriyoM/Heart-Disease-/blob/main/heart_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Make Machine Learning Model**

## i) Calling Necessary Dependencies

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

## ii) Creating DataFrame

In [ ]:
df=pd.read_csv('heart.csv')

## Note: As the main purpose of this notebook is deployment, I performed EDA and experiemnts separately and found that:
## 1)SVM, as compared to other classification algorithms works best for this problem.
## 2)Removing outliers(if the particular feature is normally distributed we use z-score method to remove the outlier, if the feature is skewed either positively or negatively we use IQR formula to remove them) has no effect on the accuracy, infact it becomes 1% less accurate.

##  iii) Separating Dependent and Independent Variables

In [ ]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
y=pd.DataFrame(df['HeartDisease'])
y=y.values.ravel()
X=df.drop(['HeartDisease'],axis=1)


## iv) Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

## v) Ordinally Encoding the Categorical Features

In [ ]:
trf1 = ColumnTransformer([
    ('oe_cat_col1',OrdinalEncoder(categories=[['F','M'],['ATA','NAP','TA','ASY'],['Normal','LVH','ST'],['N','Y'],['Up','Down','Flat']]),['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'])
],remainder='passthrough')

## vi) Scaling

In [ ]:
trf2 = ColumnTransformer([
    ('scale',StandardScaler(),[0,3,4,7,9])
],remainder='passthrough')

## vii) Modeling

In [ ]:
trf3 = svm.SVC(probability=True)

## viii)  Creating Pipeline

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3)
])

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oe_cat_col1',
                                                  OrdinalEncoder(categories=[['F',
                                                                              'M'],
                                                                             ['ATA',
                                                                              'NAP',
                                                                              'TA',
                                                                              'ASY'],
                                                                             ['Normal',
                                                                              'LVH',
                                                                              'ST'],
                                                                             ['N',
                                                                              'Y'],
                                                                             ['Up',
                                                                              'Down',
                                                                              'Flat']]),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  [0, 3, 4, 7, 9])])),
                ('trf3', SVC(probability=True))])

In [ ]:
preds=pipe.predict(X_test)

## ix) Creating Pickle Files for DataFrame and Pipeline

In [ ]:
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(pipe,open('pipe.pkl','wb'))